Analyze GPU scan parquet operator in Spark event logs.

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd

cpu_dir = "/home/rou/src/spark-rapids-tools/CPU-3k-new"
gpu_dir = "/home/rou/src/spark-rapids-tools/GPU-3k-new"
cpu_files = os.listdir(cpu_dir)
cpu_files.sort()
gpu_files = os.listdir(gpu_dir)
gpu_files.sort()

In [ ]:
gpu_scan_times = []
for i, app in enumerate(gpu_files):
    sql_info = pd.read_csv(gpu_dir + "/" + app + "/sql_plan_metrics_for_application.csv")
    filtered_df = sql_info[(sql_info["name"] == "scan time") & (sql_info["nodeName"] == "GpuScan parquet ")].copy()
    filtered_df["appIndex"] = i 
    filtered_df["app"] = app 
    filtered_df.drop_duplicates(subset=["accumulatorId"], inplace=True)
    gpu_scan_times.append(filtered_df) 
gpu_scan_times_df = pd.concat(gpu_scan_times)
gpu_scan_times_df[gpu_scan_times_df["appIndex"] == 3]

In [ ]:
cpu_scan_times = []
for i, app in enumerate(cpu_files) :
    sql_info = pd.read_csv(cpu_dir + "/" + app + "/sql_plan_metrics_for_application.csv")
    filtered_df = sql_info[(sql_info["name"] == "scan time") & (sql_info["nodeName"] == "Scan parquet ")].copy()
    filtered_df["appIndex"] = i  
    filtered_df["app"] = app 
    # filtered_df.drop_duplicates(subset=["accumulatorId"], inplace=True)
    cpu_scan_times.append(filtered_df)
cpu_scan_times_df = pd.concat(cpu_scan_times)
cpu_scan_times_df[cpu_scan_times_df["appIndex"] == 14]

In [ ]:
gpu_scan_counts = gpu_scan_times_df.groupby("appIndex").size().reset_index(name="counts gpu")
cpu_scan_counts = cpu_scan_times_df.groupby("appIndex").size().reset_index(name="counts cpu")
combined_counts = pd.merge(gpu_scan_counts, cpu_scan_counts, on="appIndex")
combined_counts[combined_counts["counts gpu"] != combined_counts["counts cpu"]]

2 - query 24a:
  CPU: 12 `Scan parquet` - 5 `ReusedExchange` = 7
  GPU: 12 `GpuScan parquet` - 4 `ReusedExchange` = 8
3 - query 24b: GPU misisng 1 `ReusedExchange`
  CPU: 12 `Scan parquet` - 5 `ReusedExchange` = 7
  GPU: 12 `GpuScan parquet` - 4 `ReusedExchange` = 8
5 - query 47:
  CPU: 15 `Scan parquet` - 11 `ReusedExchange` = 4
  GPU: 15 `GpuScan parquet` - 10 `ReusedExchange` = 5
6 - query 56:
  CPU: 18 `Scan parquet` - 9 `ReusedExchange` - 2 `ReusedSubquery` = 7
  GPU: 18 `GpuScan parquet` - 9 `ReusedExchange` - 2 `ReusedSubquery` = 7 (why 8???)
7 - query 66:
  CPU: 12 `Scan parquet` - 5 `ReusedExchange` - 1 `ReusedSubquery` = 6
  GPU: 12 `GpuScan parquet` - 4 `ReusedExchange` - 1 `ReusedSubquery` = 7
8 - query 75:
  CPU: 30 `Scan parquet` - 14 `ReusedExchange` - 4 `ReusedSubquery` = 12
  GPU: 30 `GpuScan parquet` - 12 `ReusedExchange` - 4 `ReusedSubquery` = 14  
 

14: query 14b:
  CPU: 82 `Scan parquet` - 54 `ReusedExchange` - 20 `ReusedSubquery` + 9 nested reuse = 17
  GPU: 82 `GpuScan parquet` - 64 `ReusedExchange` - 11 `ReusedSubquery` + 3 nested reuse = 10???